<a href="https://colab.research.google.com/github/thispath98/NLP/blob/main/BERT_Korean_daily_date_texts_binary_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Colab 기준으로 작성된 코드이며,
[bert_naver_movie_colab_ipynb](https://colab.research.google.com/drive/1tIf0Ugdqg4qT7gcxia3tL7und64Rv1dP#scrollTo=P58qy4--s5_x) 코드를 바탕으로 작성되었습니다. 데이터 전처리, 테스트 부분을 제외하고는 동일합니다.

[일상/연애 주제의 한국어 대화 'BERT'로 이진 분류 모델 만들기](https://velog.io/@seolini43/%EC%9D%BC%EC%83%81%EC%97%B0%EC%95%A0-%EC%A3%BC%EC%A0%9C%EC%9D%98-%ED%95%9C%EA%B5%AD%EC%96%B4-%EB%8C%80%ED%99%94-BERT%EB%A1%9C-%EC%9D%B4%EC%A7%84-%EB%B6%84%EB%A5%98-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0%ED%8C%8C%EC%9D%B4%EC%8D%ACColab-%EC%BD%94%EB%93%9C) 를 보고 실습해보았습니다.


> 해당 영역은 내가 실습하며 쓴 글이다.

# 1.Colab 환경 설정

먼저, GPU를 사용하기 위해 시스템 환경 유형 변경을 통해 `GPU`로 변경해주고, 아래 코드를 입력하여 라이브러리를 설치하고 import 해준다.

라이브러리 하는 모듈들을 보면, [이전 글](https://velog.io/@seolini43/%EC%9D%BC%EC%83%81%EC%97%B0%EC%95%A0-%EC%A3%BC%EC%A0%9C%EC%9D%98-%ED%95%9C%EA%B5%AD%EC%96%B4-%EB%8C%80%ED%99%94-BERT%EB%A1%9C-%EC%9D%B4%EC%A4%91-%EB%B6%84%EB%A5%98-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0%ED%8C%8C%EC%9D%B4%EC%8D%ACColab)에서 설명한 Bert의 Tokenizer(WordPiece), BertForSequenceClassification 등이 있는 것을 확인할 수 있다.

In [1]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 34.5 MB/s 
     |████████████████████████████████| 3.3 MB 26.2 MB/s 
     |████████████████████████████████| 56 kB 3.7 MB/s 
     |████████████████████████████████| 636 kB 29.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [3]:
# GPU 확인하기
n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
Tesla K80


# 2. 데이터셋 불러오기

## (1) 학습 데이터 불러오기

일상대화/연애대화를 분류하는 모델을 만들기 위해선, pretrain 되어 있는 BERT에 추가로 일상대화/연애대화 데이터를 학습시켜주어야 한다. 따라서 데이터를 불러오고, 입력으로 들어가기 위한 전처리가 필요하다.

한편, 추가로 학습시킬 데이터는 이전 글에서도 설명했지만 깃허브에서 가져온 [Chatbot data](https://github.com/songys/Chatbot_data)이다. 이 데이터는 11,876개의 한글 대화 문답으로 되어 있는 인공데이터로, 일상 대화, 이별과 관련된 대화, 긍정적인 사랑에 대한 대화가 각각 0, 1, 2로 라벨링 되어 있다. 이제 이 데이터를 깃허브에서 다운로드 한 뒤, 구글 드라이브를 연동하는 방식으로 데이터를 가져왔다.

> 나는 벨로그 게시물과 별개로 git clone을 사용하여 데이터를 가져왔다.

In [4]:
!git clone https://github.com/songys/Chatbot_data

Cloning into 'Chatbot_data'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 50 (delta 17), reused 2 (delta 1), pack-reused 18
Unpacking objects: 100% (50/50), done.


> 데이터 불러오고 쉐입을 확인한다.

In [5]:
import csv
import pandas as pd
chatbot_data = pd.read_csv('./Chatbot_data/ChatbotData.csv',encoding="utf-8")
chatbot_data.shape  # (11823, 3)

In [7]:
chatbot_data.sample(n=5)

,Q,A,label
5636,그녀를 만났던 카페에서 탈퇴했어,힘들겠지만 잘했어요.,1
7639,이별후에,나아지길 바랄게요.,1
3613,으으~ 떨린다,파이팅!,0
7746,이젠 붙잡을 수 도 없는 사람.,달라지는게 없다면 이런 상황이 더 나을지도 몰라요.,1
4428,착한 사람이 잘 됐으면 좋겠어 좀,뿌린대로 거둘 거예요.,0


랜덤으로 5개의 행을 출력한 결과, 위와 같이 'Q'라는 컬럼에 있는 한글 질의에 대해 'A' 컬럼에 답변 데이터가 있고, 'label' 컬럼에 해당 질의에 대해 일상 대화인지, 연애 관련 대화 중 이별(부정)에 관한 대화인지, 사랑(긍정)에 관한 대화인지 라벨링 되어 있음을 확인할 수 있다.

현재 3개의 클래스로 라벨링 되어 있는데, 이번 프로젝트에서는 일상대화인지 연애 관련 대화인지 이진분류를 할 것이기 때문에 '1'과 '2'로 라벨링 되어 있는 데이터를 모두 '1'로 바꿔주어야 한다. 따라서 아래 코드를 실행해주자.

In [8]:
chatbot_data.loc[(chatbot_data['label'] == 2), 'label'] = 1  #라벨 1과 2는 모두 1로 통일

In [11]:
display(chatbot_data.iloc[5285:5290,:])
display(chatbot_data.iloc[5291:5296,:])

,Q,A,label
5285,힘내야지,응원합니다!,0
5286,힘든 것 좀 끝났으면,다 지나갈 거예요.,0
5287,힘든 시기가 지나갔으면,다 지나갈 거예요.,0
5288,힘든 데도 날 도와준 사람,잊지말고 보답하세요.,0
5289,힘차게 시작,응원합니다!,0


,Q,A,label
5291,10년 연애. 헤어졌습니다.,더 공허함이 크시겠네요.,1
5292,10년 연애사 되돌아보니 다 부질없네,더 좋은 사람 만나실 거예요.,1
5293,10년 연예의끝,더 마음이 허하겠어요.,1
5294,10년만나다 헤어지네,충분히 슬퍼하고 충분히 아파하다가 이겨내세요.,1
5295,10년이라는 시간이 참 무색하다 싶네,함께 걸은 길을 혼자 돌아가고 계시겠네요.,1


## (2)Train set / Test set으로 나누기
데이터를 Train set과 Test set으로 나누기 전, 데이터를 확인해보면 알겠지만 0~5290 까지의 행이 모두 클래스가 '0'인 행이고, 5291행부터 마지막 행까지 모두 클래스가 '1'이다. 특정 비율로 데이터셋을 나눌 때, 클래스가 '0'인 데이터와 '1'인 데이터가 골고루 들어가기 위해선 데이터를 랜덤으로 섞어주고 나눠야 한다. 따라서 다음과 같이 코드를 입력해준다.

In [12]:
chatbot_data_shuffled = chatbot_data.sample(frac=1).reset_index(drop=True)

데이터를 섞었다면 train data와 test data의 비율을 설정해주어야 하는데, 11823개의 데이터에서 9000개는 train data로, 나머지 2823개의 데이터는 test data로 간단하게 나눠주도록 하겠다. 추후에 train data는 또 한번 train data와 validation data로 나누게 된다.



In [13]:
#train data & test data 로드 
train = chatbot_data_shuffled[:9000]
test = chatbot_data_shuffled[9000:]

print(train.shape)
print(test.shape)

(9000, 3)
(2823, 3)


In [14]:
display(train.head())
display(test.head())

,Q,A,label
0,점심시간엔 그냥 쉬고 싶어,맛난거 드시고 조금이라도 쉬셔야 할텐데.,0
1,남자친구랑 봉사활동 해보려고,의미있는 일이네요.,0
2,퇴근길 엄청 막히네,라디오를 들으며 여유를 가져보세요.,0
3,이별후 3개월 다되가는데.,마음은 좀 정리 되어가길 바랍니다.,1
4,난 나대로 걍 산다,나대로 사는게 좋죠.,1


,Q,A,label
9000,학생일 때 썸 괜찮을까,지금은 지나면 돌아오지 않아요.,1
9001,뭔가 수상한데?,해결될 거니 기다려보세요.,0
9002,나 왜 멍청해,다음에는 다를거예요.,0
9003,누군가를 좋아하는게 과연 행복한 걸까.,행복하지만은 않겠지요.,1
9004,여사친 좋아하는 내가 짜증남.,자기 자신이라도 사랑해주세요.,1


# 3.Train set 전처리
여기서부터 지금까지 전처리한 dataset이 BERT 모델의 입력데이터가 되기 위한 특수한 전처리가 필요하다.



## (1) [CLS]와 [SEP]
BERT 분류 모델의 경우 각 문장의 앞마다 [CLS]를 붙여 인식시키고, 문장의 종료는 [SEP]를 붙여 인식시킨다. [CLS]을 인식함으로써 문장의 처음이라 알 수 있게 하고, [SEP]을 인식함으로써 문장의 끝을 알 수 있게 하기 위함이다.

이전 글에서 BERT의 pretrain 방법 중 하나가 [SEP]를 인식하여 두 문장이 이어지는 문장인지, 관련 없는 문장인지 학습하는 것이었다. 아무튼 추가한 데이터 역시 학습을 해야하니 각 문장마다 [CLS]와 [SEP]를 붙여주어야 한다.

In [18]:
# CLS, SEP 붙이기 (문장의 시작, 끝)
sentences = ["[CLS] " + str(s) + " [SEP]" for s in train.Q]
sentences[:5]

['[CLS] 점심시간엔 그냥 쉬고 싶어 [SEP]',
 '[CLS] 남자친구랑 봉사활동 해보려고 [SEP]',
 '[CLS] 퇴근길 엄청 막히네 [SEP]',
 '[CLS] 이별후 3개월 다되가는데. [SEP]',
 '[CLS] 난 나대로 걍 산다 [SEP]']

한편, '0'과 '1'의 라벨이 들어있는 컬럼을 'labels'이라는 array에 따로 저장한다.

In [19]:
labels = train['label'].values
labels

array([0, 0, 0, ..., 1, 1, 1])

## (2)서브워드 토크나이저 : WordPiece
BERT에서 사용하는 토크나이저인 WordPiece에 대해 지난번 글에서 설명했다. 단어를 토큰화 할 때, 단어집합에 없는 단어는 더 쪼개서 '##'을 붙여주는 방식이었다. BertTokenizer 라이브러리를 통해 이를 수행할 수 있는데, '안녕하세요!'라는 문장에 대해 적용해보았다.

현재 단어 집합이 비어있는 상태이므로 '안녕하세요'라는 단어가 단어집합에 없다. 따라서 이 단어는 한번 더 쪼개어 지는데 '##'을 붙임으로써 '##녕', '##하', '##세', '##요' 가 어떠한 단어의 서브워드라는 것을 나타내준다. 이렇게 나타내 줌으로써 단어의 의미를 잃지 않고 다시 원래 단어로 복원을 할 수 있게 된다.

In [20]:
import pandas as pd
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", do_lower_case=False)
result = tokenizer.tokenize('안녕하세요!')
print(result)

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

['안', '##녕', '##하', '##세', '##요', '!']


그렇다면 이제 WordPiece tokenizer를 이용하여 전체 데이터에 토크나이징을 수행하도록 한다. 토크나이징을 하는 코드는 간단하다.

In [21]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(s) for s in sentences]

In [22]:
print(sentences[0])  #토크나이징 전
print(tokenized_texts[0]) #토크나이징 후

[CLS] 점심시간엔 그냥 쉬고 싶어 [SEP]
['[CLS]', '점', '##심', '##시간', '##엔', '그', '##냥', '쉬', '##고', '싶', '##어', '[SEP]']


이 이후에, 문장의 최대 시퀀스를 설정하여 정수 인코딩과 제로 패딩을 수행해준다.

In [25]:
MAX_LEN = 128 #최대 시퀀스 길이 설정
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [27]:
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
input_ids[0]

array([   101,   9668,  71013, 100699,  86933,   8924, 118729,   9469,
        11664,   9495,  12965,    102,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

## (3) Attention Mask (어텐션 마스크)
어텐션 마스크에 대해선 이전 글에서 설명을 하지 않았는데, 어텐션 마스크란 0 값을 가지는 패딩 토큰에 대해서 어텐션 연산을 불필요하게 수행하지 않도록 단어와 패딩 토큰을 구분할 수 있게 알려주는 것을 말한다.

따라서 [40311, 9435, 102, 0, 0]와 같은 패딩된 데이터가 있을 때, 패딩된 값은 '0', 패딩되지 않은 단어는 '1'의 값을 갖도록 시리얼 데이터를 만들어 주어야 한다.

    예) 패딩된 데이터 = [40311, 9435, 102, 0, 0]
    >>어텐션 마스크 = [1.0, 1.0, 1.0, 0.0, 0.0]

In [28]:
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [29]:
print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


## (4) Train set을 훈련셋과 검증셋으로 분리하기
데이터가 입력의 형태를 갖췄다면 훈련셋과 검증셋으로 분리해주도록 한다. 어텐션 마스크도 함께 훈련셋과 검증셋으로 분리하고, 데이터를 모두 파이토치 텐서로 변환시킨다.

In [30]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2000, 
                                                                                    test_size=0.1)

In [31]:
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2000, 
                                                       test_size=0.1)

In [32]:
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

마지막으로 배치사이즈를 32로 설정하고, 입력데이터, 어텐션 마스크, 라벨을 하나의 데이터로 묶어 train_dataloader, validation_dataloader라는 입력데이터를 생성해준다.

In [33]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# 4. Test set 전처리
Train set을 전처리 한 것과 같은 방식으로 Test set에도 같은 전처리 과정을 수행한다.

In [36]:
# [CLS] + 문장 + [SEP]
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in test.Q]

# 라벨 데이터
labels = test['label'].values

# Word 토크나이저 토큰화
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

# 시퀀스 설정 및 정수 인덱스 변환 & 패딩
MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# 어텐션 마스크
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

# 파이토치 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

# 배치 사이즈 설정 및 데이터 설정
batch_size = 32
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# 5. BERT 모델 불러오기
이제 BERT 모델을 불러오도록 한다. (여기서부터는 참조한 코드와 동일하다)

그 전에 GPU 설정을 위한 디바이스 설정을 해준다.

아래 코드를 실행하고 'There are 1 GPU(s) available.' 라는 문구가 출력되면 사용이 가능하다는 뜻이다.

In [37]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


이제 pretrain된 BERT 모델을 불러오도록 하겠다.

In [38]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

하이퍼 파라미터 값은 참조한 코드에 있는 값 그대로 설정했다. 에폭 수는 간단하게 4로 설정해주었다.

In [39]:
# 옵티마이저
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률(learning rate)
                  eps = 1e-8 
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)


# 6. 모델 학습
필요한 함수를 설정해주고 난 뒤, 아래 코드를 실행하여 모델을 학습시켜준다.

훈련하는 데에는 `Tesla K80` GPU 기준으로 약 20분이 소요된다.

In [40]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)
    
    
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [41]:
#랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

#그래디언트 초기화
model.zero_grad()

# 학습
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.44
  Training epcoh took: 0:05:35

Running Validation...
  Accuracy: 0.83
  Validation took: 0:00:13

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.30
  Training epcoh took: 0:05:34

Running Validation...
  Accuracy: 0.84
  Validation took: 0:00:13

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.22
  Training epcoh took: 0:05:34

Running Validation...
  Accuracy: 0.86
  Validation took: 0:00:13

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.18
  Training epcoh took: 0:05:34

Running Validation...
  Accuracy: 0.86
  Validation took: 0:00:13

Training complete!


# 7. 테스트셋 평가
지금까지 pretrain된 BERT 모델에 추가로 9000개의 한글 대화 데이터를 추가로 학습시켰다. 이제 테스트 셋 데이터로 학습시킨 모델의 정확도를 측정해보도록 하겠다.

In [42]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))


Accuracy: 0.87
Test took: 0:00:42


# 8. 새로운 문장 테스트
이제 새로운 문장을 입력하여 분류가 잘 이루어지는지 살펴보도록 하자.

입력하는 문장도 BERT의 입력 형태로 만드는 작업이 필요하기 때문에 우선 아래 함수를 정의해준다.

In [51]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [52]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [56]:
logits = test_sentences(['더 나은 학교생활 하고 싶어'])
print(logits)

if np.argmax(logits) == 1 :
    print("연애 관련 대화")
elif np.argmax(logits) == 0 :
    print("일상 대화")

[[ 2.0506775 -1.9394132]]
일상 대화


In [54]:
logits = test_sentences(['저녁 뭘 먹을지 추천해줘'])

print(logits)
if np.argmax(logits) == 1 :
    print("연애 관련 대화")
elif np.argmax(logits) == 0 :
    print("일상 대화")

[[ 1.3154305 -1.1970401]]
일상 대화


In [55]:
logits = test_sentences(['여자친구한테 선물 뭘로 줄까?'])

print(logits)
if np.argmax(logits) == 1 :
    print("연애 관련 대화")
elif np.argmax(logits) == 0 :
    print("일상 대화")

[[-1.5484756  1.33906  ]]
연애 관련 대화
